In [25]:
import pandas as pd
from get_exact_and_f1_score.ext_services.jsql_parser import JSQLParser
from get_exact_and_f1_score.metrics.partial_match_eval.evaluate import evaluate
import ast
import json

In [37]:
import re
s = "select max(credit) from master_txn_table where transaction_type = 'invoice' and instr(account,'window washing') and transaction_date between date( current_date, 'weekday 0', '-7 days') and date( current_date) "
s = re.sub(' +', ' ', str(s).replace("'", '"').lower())

In [40]:
df = pd.read_csv("generated_data/gpt4/gpt4_dynamic_few_shot_6_mod_50.csv")
df = df.apply(lambda s:s.str.replace('"', "'"))
df.iloc[72].values

array(['What was the max invoice value for Window washing in This week to date?',
       "SELECT max(credit) FROM master_txn_table WHERE transaction_type = 'invoice' AND instr(account, 'Window washing') AND Transaction_DATE >= DATE_SUB(CURDATE(), INTERVAL WEEKDAY(CURDATE()) DAY)",
       "select max(credit) from master_txn_table where transaction_type = 'invoice' and instr(account,'Window washing') and transaction_date BETWEEN date( current_date, 'weekday 0', '-7 days') AND date( current_date) "],
      dtype=object)

In [38]:
_jsql_parser = JSQLParser.create()
sql_parser1 = _jsql_parser.translate_batch(["SELECT sum(quantity), amount FROM master_txn_table WHERE customers = 'Michael Hoover' AND product_service = 'Dairy' AND trasaction_type IN ('invoice','sales receipt') AND transaction_date BETWEEN date( current_date, 'weekday 0', '-7 days') AND date( current_date)"])
sql_parser2 = _jsql_parser.translate_batch(["SELECT sum(quantity) FROM master_txn_table WHERE product_service = 'Dairy' AND customers = 'Michael Hoover' AND trasaction_type IN ('invoice','sales receipt') AND transaction_date BETWEEN date( current_date, 'weekday 0', '-7 days') AND date( current_date)"])
sql_parser3 = _jsql_parser.translate_batch([s])
sql_parser3

[{'select_body_0': [{'select_items': ['credit',
     'max',
     ['max', 'credit'],
     ['credit', 'max', ['max', 'credit']]],
    'top_items': [],
    'from_items': ['master_txn_table'],
    'where_items': ['transaction_type',
     '"invoice"',
     '=',
     [['transaction_type'], ['"invoice"'], '='],
     'instr',
     'AND',
     [['transaction_type',
       '"invoice"',
       '=',
       [['transaction_type'], ['"invoice"'], '=']],
      ['instr'],
      'AND'],
     'transaction_date',
     'AND',
     [['transaction_type',
       '"invoice"',
       '=',
       [['transaction_type'], ['"invoice"'], '='],
       'instr',
       'AND',
       [['transaction_type',
         '"invoice"',
         '=',
         [['transaction_type'], ['"invoice"'], '=']],
        ['instr'],
        'AND']],
      ['transaction_date'],
      'AND']],
    'order_items': [],
    'groupby_items': [],
    'having_items': []}]}]

In [35]:
pcm_f1_scores = evaluate(sql_parser1, sql_parser3)
pcm_em_scores = evaluate(sql_parser1, sql_parser3, exact_match=True)
pcm_f1_scores, pcm_em_scores

([0.4982817869415807], [0.0])